In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
import pandas as pd
import numpy as np

In [8]:
data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\증강된데이터_24개씩.xlsx')
data.head()

,generator_context,category1,category2,input_context,original_index,augmentation_index
0,"아침에 커피잔 옆에 작은 쪽지가 놓여 있었는데, 그 안에 따뜻한 응원의 말이 적혀 ...",기쁨,감동,설탕 스틱 껴준거 센스 백점 만점에 천점,20.0,NaN
1,"따뜻한 오후, 은퇴한 선생님이 혼자 사는 할머니 집에 손수 만든 국수를 가져다 드렸...",기쁨,감동,아쓰 산차이 기분 안 좋은 거 알아채고 산차이가 가고 싶다던 토끼집 데려온 거 감동,79.0,NaN
2,바닷가에서 맞은 노을빛은 마치 그림 속 한 장면처럼 황홀했다. 그 빛이 내 마음에 ...,기쁨,감동,신데렐라 드레스는 다시 봐도 너무 아름다워. 사람에게 꿈의 물결을 입히다니요.,104.0,NaN
3,친구가 내 생일 파티를 직접 기획하고 준비해 준 모습에 마음이 따뜻해졌어.,기쁨,감동,와 민희진 씨 애들 숙소 스타일링까지 맡기면서 신경써 준 거 진짜 좀 대단하네,107.0,NaN
4,"빗속에서 우산이 갑자기 찢어지자, 옆 사람이 자신의 우산을 건네주며 말없이 함께 걸...",기쁨,감동,개감동인 거 자기가 쓰고 있던 우산 나 주고\n자기가 비 맞아가면서 뒤집어준 거야\...,137.0,NaN


In [9]:
def embeddings_model():
  """
  임베딩 모델 초기화
  """
  model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko") 
  vec_dim = len(model.encode("dummy_text"))
  print(f"모델 차원: {vec_dim}")
  return model

In [ ]:
embeddings_model = embeddings_model()

모델 차원: 1024


In [11]:
data['vector'] = ''
data['vector'] = data['generator_context'].apply(lambda x: embeddings_model.encode(x).tolist())

In [8]:
data.head()

,generator_context,category1,category2,input_context,original_index,augmentation_index,vector
0,"아침에 커피잔 옆에 작은 쪽지가 놓여 있었는데, 그 안에 따뜻한 응원의 말이 적혀 ...",기쁨,감동,설탕 스틱 껴준거 센스 백점 만점에 천점,20.0,NaN,"[-0.0316142775118351, 0.09130116552114487, -0...."
1,"따뜻한 오후, 은퇴한 선생님이 혼자 사는 할머니 집에 손수 만든 국수를 가져다 드렸...",기쁨,감동,아쓰 산차이 기분 안 좋은 거 알아채고 산차이가 가고 싶다던 토끼집 데려온 거 감동,79.0,NaN,"[0.014902042225003242, 0.05517202988266945, 0...."
2,바닷가에서 맞은 노을빛은 마치 그림 속 한 장면처럼 황홀했다. 그 빛이 내 마음에 ...,기쁨,감동,신데렐라 드레스는 다시 봐도 너무 아름다워. 사람에게 꿈의 물결을 입히다니요.,104.0,NaN,"[-0.033161792904138565, 0.016575686633586884, ..."
3,친구가 내 생일 파티를 직접 기획하고 준비해 준 모습에 마음이 따뜻해졌어.,기쁨,감동,와 민희진 씨 애들 숙소 스타일링까지 맡기면서 신경써 준 거 진짜 좀 대단하네,107.0,NaN,"[0.042953308671712875, 0.07023525983095169, 0...."
4,"빗속에서 우산이 갑자기 찢어지자, 옆 사람이 자신의 우산을 건네주며 말없이 함께 걸...",기쁨,감동,개감동인 거 자기가 쓰고 있던 우산 나 주고\n자기가 비 맞아가면서 뒤집어준 거야\...,137.0,NaN,"[-0.005945381708443165, 0.07036446779966354, 0..."


In [ ]:
data['vector'] = data['vector'].apply(lambda x: np.array(x) if isinstance(x, list) else np.array(eval(x)))


X = np.vstack(data['vector'].values)   # (N, 1024)
y = data['category1'].values           # (N,)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
print("X shape:", X.shape)   # (샘플 수, 1024)
print("y shape:", y.shape)   # (샘플 수,)

X shape: (1680, 1024)
y shape: (1680,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",  # GPU 있으면 "gpu_hist"
    random_state=42
)

clf.fit(X_train, y_train)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

          기쁨       0.83      0.94      0.88        72
         두려움       0.81      0.92      0.86        24
     미움(상대방)       0.77      0.89      0.83        38
          분노       0.91      0.72      0.81        29
          사랑       0.88      0.82      0.85        34
         수치심       0.65      0.79      0.71        14
          슬픔       0.79      0.79      0.79        58
     싫어함(상태)       0.57      0.54      0.55        24
          욕망       0.69      0.62      0.65        29
          중립       1.00      0.29      0.44        14

    accuracy                           0.79       336
   macro avg       0.79      0.73      0.74       336
weighted avg       0.80      0.79      0.78       336



,generator_context,category1,category2,input_context,original_index,augmentation_index,vector


In [12]:
# 문제 해결: 벡터 컬럼이 이미 리스트 형태로 잘 되어 있음
print("데이터 상태 확인:")
print(f"데이터 개수: {len(data)}")
print(f"벡터 타입: {type(data['vector'].iloc[0])}")
print(f"벡터 길이: {len(data['vector'].iloc[0])}")

# 벡터가 이미 리스트 형태라면 직접 numpy array로 변환
if isinstance(data['vector'].iloc[0], list):
    print("벡터가 리스트 형태입니다. 직접 변환합니다...")
    X = np.vstack(data['vector'].values)
    y = data['category1'].values
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print("✅ 성공적으로 변환되었습니다!")
else:
    print("벡터 형태에 문제가 있습니다.")

데이터 상태 확인:
데이터 개수: 1680
벡터 타입: <class 'list'>
벡터 길이: 1024
벡터가 리스트 형태입니다. 직접 변환합니다...
X shape: (1680, 1024)
y shape: (1680,)
✅ 성공적으로 변환되었습니다!


In [13]:
# 올바른 방법으로 모델 학습하기

# 1. 데이터가 있는지 확인
if len(data) == 0:
    print("⚠️ 데이터가 비어있습니다! 위 셀을 먼저 실행해주세요.")
else:
    # 2. 벡터와 라벨 준비
    if isinstance(data['vector'].iloc[0], list):
        X = np.vstack(data['vector'].values)
        y = data['category1'].values
        
        print(f"✅ 데이터 준비 완료:")
        print(f"  - X shape: {X.shape}")
        print(f"  - y shape: {y.shape}")
        
        # 3. 라벨 인코딩
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        
        # 4. 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )
        
        # 5. XGBoost 모델 학습
        print("\n🚀 모델 학습 시작...")
        model = xgb.XGBClassifier(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=8,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            tree_method="hist",
            n_jobs=-1
        )
        
        model.fit(X_train, y_train)
        print("✅ 학습 완료!")
        
        # 6. 평가
        y_pred = model.predict(X_test)
        
        print("\n📊 성능 평가:")
        print(classification_report(y_test, y_pred, target_names=le.classes_))
        
        # 7. 정확도
        accuracy = (y_pred == y_test).mean()
        print(f"\n🎯 전체 정확도: {accuracy:.4f}")

✅ 데이터 준비 완료:
  - X shape: (1680, 1024)
  - y shape: (1680,)

🚀 모델 학습 시작...
✅ 학습 완료!

📊 성능 평가:
              precision    recall  f1-score   support

          기쁨       0.83      0.94      0.88        72
         두려움       0.81      0.92      0.86        24
     미움(상대방)       0.77      0.89      0.83        38
          분노       0.91      0.72      0.81        29
          사랑       0.88      0.82      0.85        34
         수치심       0.65      0.79      0.71        14
          슬픔       0.79      0.79      0.79        58
     싫어함(상태)       0.57      0.54      0.55        24
          욕망       0.69      0.62      0.65        29
          중립       1.00      0.29      0.44        14

    accuracy                           0.79       336
   macro avg       0.79      0.73      0.74       336
weighted avg       0.80      0.79      0.78       336


🎯 전체 정확도: 0.7887
